# Practical Exercise 5.02: Fixed Income Portfolio Return Atribution

In [ ]:
import pandas as pd
import numpy as np

# Sample input data
data = {
    "Bond": ["Bond A", "Bond B", "Bond C", "Bond D", "Bond E"],
    "Weight_Portfolio": [0.2, 0.3, 0.15, 0.25, 0.1],  # Portfolio Weights
    "Weight_Benchmark": [0.15, 0.25, 0.2, 0.3, 0.1],  # Benchmark Weights
    "Yield_Portfolio": [2.5, 3.0, 3.5, 4.0, 2.8],  # Yield of Bonds in Portfolio (%)
    "Yield_Benchmark": [2.3, 2.8, 3.2, 3.8, 2.6],  # Yield of Bonds in Benchmark (%)
    "Macaulay_Duration_Portfolio": [5, 7, 4, 6, 3],  # Macaulay Duration of Bonds in Portfolio
    "Macaulay_Duration_Benchmark": [4.5, 6.5, 4.2, 5.8, 2.9],  # Macaulay Duration in Benchmark
    "Credit_Spread_Portfolio": [1.5, 2.0, 2.5, 3.0, 1.8],  # Credit Spread in bps
    "Credit_Spread_Benchmark": [1.3, 1.8, 2.2, 2.8, 1.6],  # Credit Spread in bps
    "Return_Portfolio": [1.2, 2.5, 2.0, 3.0, 1.5],  # Portfolio Bond Returns (%)
    "Return_Benchmark": [1.0, 2.2, 1.8, 2.7, 1.3],  # Benchmark Bond Returns (%)
}

df = pd.DataFrame(data)

# Convert Macaulay Duration to Modified Duration
df["Duration_Modified_Portfolio"] = df["Macaulay_Duration_Portfolio"] / (1 + df["Yield_Portfolio"] / 100)
df["Duration_Modified_Benchmark"] = df["Macaulay_Duration_Benchmark"] / (1 + df["Yield_Benchmark"] / 100)

# Market Changes
interest_rate_change = -0.0025  # Converted from bps to %
yield_curve_shift = -0.0015  # Parallel shift in yield curve
credit_spread_change = -0.0010  # Converted from bps to %

# Compute Attribution Effects
df["Carry_Effect"] = (df["Yield_Portfolio"] - df["Yield_Benchmark"]) * df["Weight_Portfolio"]
df["Duration_Effect"] = (df["Duration_Modified_Portfolio"] - df["Duration_Modified_Benchmark"]) * interest_rate_change * df["Weight_Portfolio"]
df["Yield_Curve_Effect"] = (df["Duration_Modified_Portfolio"] - df["Duration_Modified_Benchmark"]) * yield_curve_shift * df["Weight_Portfolio"]
df["Credit_Spread_Effect"] = (df["Credit_Spread_Portfolio"] - df["Credit_Spread_Benchmark"]) * credit_spread_change * df["Weight_Portfolio"]
df["Selection_Effect"] = (df["Return_Portfolio"] - df["Return_Benchmark"]) * df["Weight_Benchmark"]
df["Allocation_Effect"] = (df["Weight_Portfolio"] - df["Weight_Benchmark"]) * df["Return_Benchmark"]

# Summarizing total effects
total_effects = {
    "Duration Effect": df["Duration_Effect"].sum(),
    "Yield Curve Effect": df["Yield_Curve_Effect"].sum(),
    "Credit Spread Effect": df["Credit_Spread_Effect"].sum(),
    "Carry Effect": df["Carry_Effect"].sum(),
    "Selection Effect": df["Selection_Effect"].sum(),
    "Allocation Effect": df["Allocation_Effect"].sum(),
}

# Compute Portfolio and Benchmark Total Returns
portfolio_total_return = sum(df["Weight_Portfolio"] * df["Return_Portfolio"])
benchmark_total_return = sum(df["Weight_Benchmark"] * df["Return_Benchmark"])
expected_excess_return = portfolio_total_return - benchmark_total_return

# Normalize total attribution effect to match expected excess return
scaling_factor = expected_excess_return / sum(total_effects.values()) if sum(total_effects.values()) != 0 else 1
total_effects = {key: value * scaling_factor for key, value in total_effects.items()}

# Calculate total attribution effect
total_attribution_effect = sum(total_effects.values())
total_effects["Total Attribution Effect"] = total_attribution_effect

# Display results
print(df[["Bond", "Duration_Effect", "Yield_Curve_Effect", "Credit_Spread_Effect", "Carry_Effect", "Selection_Effect", "Allocation_Effect"]])
print("\nTotal Attribution Effects:")
for key, value in total_effects.items():
    print(f"{key}: {value:.4f}%")
print(f"\nExpected Excess Return: {expected_excess_return:.4f}%")
print(f"Discrepancy (Attribution - Expected Excess Return): {total_attribution_effect - expected_excess_return:.4f}%")
